<a href="https://colab.research.google.com/github/ShuvamAich/Temporal-Driver-Activity-Detection/blob/Human-detection-YOLOv5/Driver-detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=cbd842ce3b7d7bd0c57b7058176d03d4b526623eb1c6b4c2da6b6dbe16e19d75
  Stored in di

In [ ]:
pip install opencv-python

In [ ]:
import cv2
import torch
from pathlib import Path
import warnings

# Suppress FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)



model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

input_video_path = '/content/run1b_2018-05-29-14-02-47.ids_2 - Trim.mp4'
output_video_path = '/content/human_detection_output.mp4'

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-12-19 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# Open the input video
cap = cv2.VideoCapture(input_video_path)

if not cap.isOpened():
    print(f"Error: Unable to open video file {input_video_path}")
    exit()

# Get video properties
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video properties: {original_width}x{original_height} @ {fps} FPS, {total_frames} total frames")

# Resize settings for faster processing
resize_width = 640
resize_height = 360

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (resize_width, resize_height))

# Frame skipping for speedup
frame_skip = 2  # Process every 2nd frame
frame_count = 0
processed_count = 0

batch = []
batch_size = 4  # Batch size for YOLO inference

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    # Resize frame for faster processing
    frame = cv2.resize(frame, (resize_width, resize_height))
    batch.append(frame)

    print(f"Processing frame {frame_count}/{total_frames}")  # Display the current frame being processed

    if len(batch) == batch_size:
        # Perform batch inference
        results = model(batch)

        for i, result in enumerate(results.xyxy):
            detections = result.cpu().numpy()  # [x1, y1, x2, y2, confidence, class]
            for *box, confidence, cls in detections:
                if int(cls) == 0:  # Class 0 corresponds to 'person'
                    x1, y1, x2, y2 = map(int, box)
                    label = f"Person {confidence:.2f}"
                    cv2.rectangle(batch[i], (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(batch[i], label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            # Write processed frame to output video
            out.write(batch[i])
            processed_count += 1

        batch = []  # Clear batch

# Release resources
cap.release()
out.release()
print(f"Detection complete. Processed {processed_count} frames out of {total_frames}. Output saved at {output_video_path}")

Video properties: 1280x1024 @ 30 FPS, 1828 total frames
Processing frame 2/1828
Processing frame 4/1828
Processing frame 6/1828
Processing frame 8/1828
Processing frame 10/1828
Processing frame 12/1828
Processing frame 14/1828
Processing frame 16/1828
Processing frame 18/1828
Processing frame 20/1828
Processing frame 22/1828
Processing frame 24/1828
Processing frame 26/1828
Processing frame 28/1828
Processing frame 30/1828
Processing frame 32/1828
Processing frame 34/1828
Processing frame 36/1828
Processing frame 38/1828
Processing frame 40/1828
Processing frame 42/1828
Processing frame 44/1828
Processing frame 46/1828
Processing frame 48/1828
Processing frame 50/1828
Processing frame 52/1828
Processing frame 54/1828
Processing frame 56/1828
Processing frame 58/1828
Processing frame 60/1828
Processing frame 62/1828
Processing frame 64/1828
Processing frame 66/1828
Processing frame 68/1828
Processing frame 70/1828
Processing frame 72/1828
Processing frame 74/1828
Processing frame 76/182

In [ ]:
from IPython.display import display, Video
# Display the output video inline in Colab
print("Evaluating the processed output video:")
display(Video(output_video_path, embed=True, width=640, height=360))



Evaluating the processed output video:
